In [69]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from tqdm import tqdm 

In [70]:
data_train = pd.read_csv('../ML/datasets/adult_data_train.csv')
data = pd.read_csv('../ML/datasets/adult_data_reserved.csv')

In [4]:
data_train.shape

(26048, 15)

In [38]:
data_train.replace('?', pd.NA, inplace=True)
data.replace('?', pd.NA, inplace=True)

In [8]:
data_train.shape

(26048, 15)

In [39]:
# data_train.dropna(inplace=True)
# data_train.reset_index(drop=True, inplace=True)
data_train.fillna(data.mode().iloc[0], inplace=True)
data.shape

(6513, 14)

In [71]:
data_train

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,32,Private,37210,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,1
1,43,Private,101950,Masters,14,Never-married,Exec-managerial,Not-in-family,White,Female,0,0,45,United-States,0
2,20,?,122244,HS-grad,9,Never-married,?,Not-in-family,White,Female,0,0,28,United-States,0
3,40,Local-gov,24763,Some-college,10,Divorced,Transport-moving,Unmarried,White,Male,6849,0,40,United-States,0
4,24,Private,113936,Bachelors,13,Never-married,Prof-specialty,Own-child,White,Male,0,0,40,United-States,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26043,35,Self-emp-not-inc,179171,HS-grad,9,Never-married,Sales,Unmarried,Black,Female,0,0,38,Germany,0
26044,39,Private,421633,Some-college,10,Divorced,Protective-serv,Unmarried,Black,Female,0,0,30,United-States,0
26045,55,Private,82098,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,Asian-Pac-Islander,Male,0,0,55,United-States,0
26046,26,Private,50053,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,55,United-States,0


In [41]:
data.fillna(data.mode().iloc[0], inplace=True)

In [54]:
data['age'].mode()[0]

33

In [24]:
for column in tqdm(data.columns):
    # print(f'Processing column: {column}')
    for i, value in enumerate(data[column]):
      if value not in data_train[column].values:
        most_frequent_value = data[column].mode()[0]
        data.at[i, column] = most_frequent_value
data.columns

100%|██████████| 14/14 [00:30<00:00,  2.16s/it]


Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country'],
      dtype='object')

In [55]:
data

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,31,117963,16,0,0,40,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1,20,201490,10,0,0,40,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
2,78,385242,13,9386,0,45,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
3,32,164197,13,0,0,44,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4,61,92691,9,0,0,3,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6508,42,32185,13,0,0,60,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
6509,34,180714,10,0,2179,40,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
6510,39,202027,15,0,0,45,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
6511,29,125976,9,0,0,35,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


In [14]:
data = data.drop(columns=['education', 'marital-status'])

In [15]:
data_train = data_train.drop(columns=['education', 'marital-status'])

In [16]:
data.shape

(6513, 12)

In [ ]:
le = LabelEncoder()
str_columns = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
for i in str_columns:
	le.fit(data_train[i])
	data_train[i] = le.transform(data_train[i])
for i in str_columns:
	le.fit(data[i])
	data[i] = le.transform(data[i])

In [72]:
data_train = pd.get_dummies(data_train, drop_first=True)

In [73]:
data = pd.get_dummies(data, drop_first=True)

In [74]:
data_train = data_train.drop(columns='native-country_Holand-Netherlands')

In [81]:
data_train

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,label,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,32,37210,13,0,0,45,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,43,101950,14,0,0,45,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,20,122244,9,0,0,28,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,40,24763,10,6849,0,40,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4,24,113936,13,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26043,35,179171,9,0,0,38,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26044,39,421633,10,0,0,30,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
26045,55,82098,9,0,0,55,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
26046,26,50053,13,0,0,55,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [75]:
X = data_train.drop(columns = 'label')
y = data_train['label']

In [76]:
scaler = MinMaxScaler()
scaler.fit(X)
X_train_scaler = scaler.transform(X)
X_test_scaler = scaler.transform(data)

In [77]:
knn = KNeighborsClassifier()
knn.fit(X_train_scaler, y)
y_pred = knn.predict(X_test_scaler)
y_pred.shape

(6513,)

In [32]:
params = {
	'n_neighbors': np.arange(1, 16), 
	'metric': ['manhattan', 'euclidean'],
	'weights': ['uniform', 'distance']
}
knn_grid = GridSearchCV(knn, params, cv=5, scoring='f1', n_jobs=-1)
knn_grid.fit(X_train_scaler, y)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'metric': ['manhattan', 'euclidean'],
                         'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]),
                         'weights': ['uniform', 'distance']},
             scoring='f1')

In [78]:
knn_grid.best_params_

{'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'uniform'}

In [79]:
knn = KNeighborsClassifier(n_neighbors=9)
knn.fit(X_train_scaler, y)
y_pred = knn.predict(X_test_scaler)
y_pred.shape

(6513,)

In [80]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier()
rfc.fit(X_train_scaler, y)
y_pred = rfc.predict(X_train_scaler)

In [52]:
with open('../ML/outputs/file_adultReserve.txt', 'w') as f:
    f.write(f'{list(y_pred)}')